In [1]:
from statsbombpy import sb
import pandas as pd
import numpy as np
from mplsoccer import VerticalPitch,Pitch, Sbopen
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
free_comps = sb.competitions()
wsl_20_21 = sb.matches(competition_id = 37, season_id = 90).sort_values(by = "match_date").reset_index(drop = True)
wsl_20_21.columns
wsl_20_21[wsl_20_21["match_id"] == 3764233]
wsl_test = wsl_20_21.head(5).copy()
wsl_test

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3764234,2020-09-05,15:30:00.000,England - FA Women's Super League,2020/2021,Aston Villa,Manchester City WFC,0,2,available,...,2021-06-13T16:17:31.694,1,Regular Season,Villa Park,Abigail Byrne,Gemma Davies,Gareth Taylor,1.1.0,2,2
1,3764231,2020-09-06,15:00:00.000,England - FA Women's Super League,2020/2021,Brighton & Hove Albion WFC,Birmingham City WFC,2,0,available,...,2021-06-13T16:17:31.694,1,Regular Season,Broadfield Stadium,Rob Whitton,Hope Patricia Powell,Carla Ward,1.1.0,2,2
2,3764230,2020-09-06,15:00:00.000,England - FA Women's Super League,2020/2021,Tottenham Hotspur Women,West Ham United LFC,1,1,available,...,2021-06-13T16:17:31.694,1,Regular Season,The Hive Stadium,Kirsty Dowle,Karen Hills,Matt Beard,1.1.0,2,2
3,3764232,2020-09-06,15:30:00.000,England - FA Women's Super League,2020/2021,Manchester United,Chelsea FCW,1,1,available,...,2021-06-13T16:17:31.694,1,Regular Season,Leigh Sports Village Stadium,Rebecca Welch,Casey Stoney,Emma Hayes,1.1.0,2,2
4,3764233,2020-09-06,13:30:00.000,England - FA Women's Super League,2020/2021,Arsenal WFC,Reading WFC,6,1,available,...,2021-06-13T16:17:31.694,1,Regular Season,Meadow Park,Amy Fearn,Joseph Montemurro,Kelly Chambers,1.1.0,2,2


In [3]:
def calculate_possession(match_df):
    home_team = match_df["team"][0]
    away_team = match_df["team"][1]
    home_team_pos = 0
    away_team_pos = 0
    for index, row in match_df.iterrows():
        if row["possession_team"] == home_team:
            home_team_pos +=1
        elif row["possession_team"] == away_team:
            away_team_pos +=1
        else:
            continue
    return [round(home_team_pos/(home_team_pos + away_team_pos),2)*100, round(away_team_pos/(home_team_pos + away_team_pos),2)*100]
#home_pos = calculate_possession(test_match)[0]
#a#way_pos = calculate_possession(test_match)[1]
#print(home_pos, away_pos)

In [103]:
match_data_dict = {}
touch_events = ['Pass', 'Dribble', 'Shot', 'Interception', 'Clearance', 'Ball Recovery']
def match_data_extract(match_df, dictionary):
        for index, row in match_df.iterrows():
        # Store matchday information for each match_id in the dictionary
            dictionary[f"{row["match_date"]}:{row["home_team"]} vs. {row["away_team"]}"] = {}
            match_id = row["match_id"]
            match_date = row["match_date"]
            home_team = row["home_team"]
            away_team = row["away_team"]
            home_score = row["home_score"]
            away_score = row["away_score"]
            match_result = "Home Win" if home_score > away_score else "Away Win" if away_score > home_score else "Draw"
            winning_team = home_team if match_result == "Home Win" else away_team if match_result == "Away Win" else "Draw"
            game = sb.events(match_id = match_id)
            game["minute"] = game["minute"] + 1 # Convert to ordered time i.e: minute = 80 - event occurred in 81st minute
            game[["location_x", "location_y"]] = game["location"].apply(pd.Series)
            game[["pass_end_location_x", "pass_end_location_y"]] = game["pass_end_location"].apply(pd.Series)
            game["pass_distance"] = round(np.sqrt((game["pass_end_location_x"] - game["location_x"])**2 + (game["pass_end_location_y"]- game["location_y"])**2))
            # Extract count based stats, home, away, and total

            # Half-time score
            first_half_home = game.loc[(game["team"] == home_team) & (game["period"] == 1)]
            first_half_away = game.loc[(game["team"] == away_team) & (game["period"] == 1)]

            half_time_score_home = first_half_home.loc[(first_half_home["type"] == "Shot") & (first_half_home["shot_outcome"] == "Goal")].shape[0]
            half_time_score_away = first_half_away.loc[(first_half_away["type"] == "Shot") & (first_half_away["shot_outcome"] == "Goal")].shape[0]
            
            # Clearances
            # Unusually large values for clearances
            # Count adjusted to include only 1 clearance for each unique possession
            # Goal kicks and free kicks not included in clearance count            
            home_clearances = game.loc[(game["team"] == home_team) & (game["type"] == "Clearance")].drop_duplicates('possession')
            home_clearances = home_clearances[~home_clearances["play_pattern"].isin(['From Goal Kick', 'From Free Kick'])].shape[0]
            away_clearances = game.loc[(game["team"] == away_team) & (game["type"] == "Clearance")].drop_duplicates('possession')
            away_clearances = away_clearances[~away_clearances["play_pattern"].isin(['From Goal Kick', 'From Free Kick'])].shape[0]
            total_clearances = home_clearances + away_clearances
            
            # Shots
            home_shots = game.loc[(game["team"] == home_team) & (game["type"] == "Shot")].shape[0]
            away_shots = game.loc[(game["team"] == away_team) & (game["type"] == "Shot")].shape[0]
            total_shots = home_shots + away_shots
            
            # Shots on Target
            home_SoT = game.loc[(game["team"] == home_team) & (game["type"] == "Shot")]
            home_SoT = home_SoT[~home_SoT["shot_outcome"].isin(["Blocked", "Wayward", "Off T"])].shape[0]
            away_SoT = game.loc[(game["team"] == away_team) & (game["type"] == "Shot")]
            away_SoT = away_SoT[~away_SoT["shot_outcome"].isin(["Blocked", "Wayward", "Off T"])].shape[0]
            total_SoT = home_SoT + away_SoT

            # Goal times
            # Identify goals for each team and extract a list of the minute values
            home_goal_times = game.loc[(game["team"] == home_team) & (game["shot_outcome"] == "Goal")]["minute"].tolist()
            away_goal_times = game.loc[(game["team"] == away_team) & (game["shot_outcome"] == "Goal")]["minute"].tolist()
            all_goal_times = home_goal_times + away_goal_times
            
            # Possession %
            home_possession = calculate_possession(game)[0]
            away_possession = calculate_possession(game)[1]
            
            # Corners
            home_corners = game.loc[(game["team"] == home_team) & (game["pass_type"] == "Corner")].shape[0]
            away_corners = game.loc[(game["team"] == away_team) & (game["pass_type"] == "Corner")].shape[0]
            total_corners = home_corners + away_corners
            
            #Fouls Conceded
            home_fouls_committed = game.loc[(game["team"] == home_team) & (game["type"] == "Foul Committed")].shape[0]
            away_fouls_committed = game.loc[(game["team"] == away_team) & (game["type"] == "Foul Committed")].shape[0]
            total_fouls = home_fouls_committed + away_fouls_committed
            
            #Offsides
            home_offside = game.loc[(game["team"] == home_team) & (game["type"] == "Offside")].shape[0]
            away_offside = game.loc[(game["team"] == away_team) & (game["type"] == "Offside")].shape[0]
            total_offside = home_offside + away_offside
            
            #Passes
            home_passes = game.loc[(game["team"] == home_team) & (game["type"] == "Pass") & (game["pass_outcome"].isnull())]
            away_passes = game.loc[(game["team"] == away_team) & (game["type"] == "Pass") & (game["pass_outcome"].isnull())]

            home_passes_count = home_passes.shape[0]
            away_passes_count = away_passes.shape[0]
            total_passes_count = home_passes_count + away_passes_count

            # Passes into 18 yard box (18 - 62 y)
            home_passes_into_box = home_passes.loc[(home_passes["pass_end_location_x"] > 102) & ((home_passes["pass_end_location_y"] > 18) & (home_passes["pass_end_location_y"] < 62))].shape[0]
            away_passes_into_box = away_passes.loc[(away_passes["pass_end_location_x"] > 102) & ((home_passes["pass_end_location_y"] > 18) & (home_passes["pass_end_location_y"] < 62))].shape[0]
            total_passes_into_box = home_passes_into_box + away_passes_into_box

            # Calculate subsets of pass length 
            home_passes_short = home_passes[home_passes["pass_distance"] < 15].shape[0]
            home_passes_medium = home_passes[(home_passes["pass_distance"] < 30) & (home_passes["pass_distance"] >15)].shape[0] 
            home_passes_long = home_passes[home_passes["pass_distance"] > 30].shape[0]

            away_passes_short = away_passes[away_passes["pass_distance"] < 15].shape[0]
            away_passes_medium = away_passes[(away_passes["pass_distance"] < 30) & (away_passes["pass_distance"] >15)].shape[0]
            away_passes_long = away_passes[away_passes["pass_distance"] > 30].shape[0]

            home_average_pass_length = round(np.mean(home_passes["pass_distance"]),2)
            away_average_pass_length = round(np.mean(away_passes["pass_distance"]),2)
            
            #Tackles
            home_tackles = game.loc[(game["team"] == home_team) & (game["duel_type"] == "Tackle") & (game["duel_outcome"] == "Won")].shape[0]
            away_tackles = game.loc[(game["team"] == away_team) & (game["duel_type"] == "Tackle") & (game["duel_outcome"] == "Won")].shape[0]
            total_tackles = home_tackles + away_tackles
            
            #Touches
            home_touches = game.loc[(game["team"] == home_team) & (game["type"].isin(touch_events))].drop_duplicates("id").shape[0]
            away_touches = game.loc[(game["team"] == away_team) & (game["type"].isin(touch_events))].drop_duplicates("id").shape[0]
            total_touches = home_touches + away_touches
            
            # Yellow and Red Cards

            # Identify rows/ columns where a Yellow/ Red card is listed
            yellows = game[game.apply(lambda col: col.astype(str).str.contains('Yellow Card')).any(axis=1)] # https://stackoverflow.com/a/78718132
            second_yellows = game[game.apply(lambda col: col.astype(str).str.contains('Second Yellow')).any(axis=1)]
            reds = game[(game.apply(lambda col: col.astype(str).str.contains('Red Card')).any(axis=1)) | \
            (game.apply(lambda col: col.astype(str).str.contains('Second Yellow')).any(axis=1))]

            # Combine all yellow card rows for home_team
            home_yellows = yellows.loc[yellows["team"] == home_team] + second_yellows.loc[second_yellows["team"] == home_team]
            # Combine all yellow card rows for away_team
            away_yellows = yellows.loc[yellows["team"] == away_team] + second_yellows.loc[second_yellows["team"] == away_team]
            
            # Subset red card rows for each team
            home_reds = reds.loc[reds["team"] == home_team] 
            away_reds = reds.loc[reds["team"] == away_team]

        
            # Extract row counts for each card type per team
            home_yellows_int = home_yellows.shape[0]
            away_yellows_int = away_yellows.shape[0]
            home_reds_int = home_reds.shape[0] 
            away_reds_int = away_reds.shape[0]
    
            total_yellows = home_yellows_int + away_yellows_int
            total_reds = home_reds_int + away_reds_int

            # Extract a list of times for each yellow card for the home_team
            home_first_yellow_times = yellows.loc[yellows["team"] == home_team]["minute"].tolist()
            home_second_yellow_times = second_yellows.loc[second_yellows["team"] == home_team]["minute"].tolist()
            home_yellow_times = home_first_yellow_times + home_second_yellow_times

            # Extract a list of times for each yellow card for the away_team
            away_first_yellow_times = yellows.loc[yellows["team"] == away_team]["minute"].tolist()
            away_second_yellow_times = second_yellows.loc[second_yellows["team"] == away_team]["minute"].tolist()
            away_yellow_times = away_first_yellow_times + away_second_yellow_times

            # Extract lists of the times when a red card was issued
            home_red_times = home_reds["minute"].tolist()
            away_red_times = away_reds["minute"].tolist()

            # Combine the lists from each team for all cards
            all_yellow_times = home_yellow_times + away_yellow_times
            all_red_times = home_red_times + away_red_times
            
            # Build a nested dictionary of all games
            dictionary[f"{row["match_date"]}:{row["home_team"]} vs. {row["away_team"]}"].update({
                "match_id": match_id,
                "match_date": match_date,
                "match_result": match_result,
                "home_team": home_team,
                "away_team": away_team,
                "winning team": winning_team,
                "half_time_score_home": half_time_score_home,
                "half_time_score_away": half_time_score_away,
                "home_goal_times": home_goal_times,
                "away_goal_times": away_goal_times,
                "all_goal_times": all_goal_times,
                "home_clearances": home_clearances,
                "away_clearances": away_clearances,
                "total_clearances": total_clearances,
                "home_shots": home_shots,
                "away_shots": away_shots,
                "home_shot_on_target": home_SoT,
                "away_shot_on_target": away_SoT,
                "total_shot_on_target": total_SoT,
                "home_possession": home_possession,
                "away_possession": away_possession,
                "home_corners": home_corners,
                "away_corners": away_corners, 
                "total_corners": total_corners,
                "home_fouls_committed": home_fouls_committed,
                "away_fouls_committed": away_fouls_committed,
                "total_fouls": total_fouls,
                "home_offsides": home_offside,
                "away_offside": away_offside,
                "total_offside": total_offside,
                "home_passes": home_passes_count,
                "away_passes": away_passes_count,
                "total_passes": total_passes_count,
                "home_passes_into_box": home_passes_into_box,
                "away_passes_into_box": away_passes_into_box,
                "total_passes_into_box": total_passes_into_box,
                "home_passes_short": home_passes_short,
                "home_passes_medium": home_passes_medium, 
                "home_passes_long": home_passes_long, 
                "away_passes_short": away_passes_short, 
                "away_passes_medium": away_passes_medium, 
                "away_passes_long": away_passes_long,
                "home_average_pass_length": home_average_pass_length,
                "away_average_pass_length": away_average_pass_length,
                "home_tackles": home_tackles,
                "away_tackles": away_tackles,
                "total_tackles": total_tackles,
                "home_touches": home_touches,
                "away_touches": away_touches,
                "total_touches": total_touches,
                "home_yellow_cards": home_yellows_int,
                "away_yellow_cards": away_yellows_int,
                "total_yellow_cards": total_yellows,
                "home_red_cards": home_reds_int,
                "away_red_cards": away_reds_int,
                "total_red_cards": total_reds,
                "home_yellow_times": home_yellow_times,
                "away_yellow_times": away_yellow_times,
                "all_yellow_times": all_yellow_times,
                "home_red_times": home_red_times,
                "away_red_times": away_red_times,
                "all_red_times": all_red_times,
            })


In [104]:
import time

# Assuming you have your match_df, dictionary, and columns defined
start_time = time.time()

# Call your function
# Just extract events based totals and merge with match list table      
match_data_extract(wsl_20_21, match_data_dict)
match_data_dict
data = []
for k in match_data_dict.keys():
    data.append(match_data_dict[k])

df = pd.DataFrame(data)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Execution time: {elapsed_time:.2f} seconds")

Execution time: 711.10 seconds


In [109]:
pd.set_option("display.max_columns", None)
df
full_season = pd.merge(wsl_20_21, df, on = ["match_id", "home_team", "away_team", "match_date"])
full_season = full_season.drop(['match_status', 'match_status_360', 'last_updated', 'last_updated_360', 'data_version', 'shot_fidelity_version',
       'xy_fidelity_version'], axis = 1)
full_season.head(8)

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_week,competition_stage,stadium,referee,home_managers,away_managers,match_result,winning team,half_time_score_home,half_time_score_away,home_goal_times,away_goal_times,all_goal_times,home_clearances,away_clearances,total_clearances,home_shots,away_shots,home_shot_on_target,away_shot_on_target,total_shot_on_target,home_possession,away_possession,home_corners,away_corners,total_corners,home_fouls_committed,away_fouls_committed,total_fouls,home_offsides,away_offside,total_offside,home_passes,away_passes,total_passes,home_passes_into_box,away_passes_into_box,total_passes_into_box,home_passes_short,home_passes_medium,home_passes_long,away_passes_short,away_passes_medium,away_passes_long,home_average_pass_length,away_average_pass_length,home_tackles,away_tackles,total_tackles,home_touches,away_touches,total_touches,home_yellow_cards,away_yellow_cards,total_yellow_cards,home_red_cards,away_red_cards,total_red_cards,home_yellow_times,away_yellow_times,all_yellow_times,home_red_times,away_red_times,all_red_times
0,3764234,2020-09-05,15:30:00.000,England - FA Women's Super League,2020/2021,Aston Villa,Manchester City WFC,0,2,1,Regular Season,Villa Park,Abigail Byrne,Gemma Davies,Gareth Taylor,Away Win,Manchester City WFC,0,2,[],"[6, 21]","[6, 21]",18,6,24,7,17,2,6,8,37.0,63.0,2,8,10,6,11,17,0,1,1,290,547,837,4,0,4,100,132,48,195,259,60,20.59,19.05,5,10,15,506,778,1284,0,0,0,0,0,0,[],[],[],[],[],[]
1,3764231,2020-09-06,15:00:00.000,England - FA Women's Super League,2020/2021,Brighton & Hove Albion WFC,Birmingham City WFC,2,0,1,Regular Season,Broadfield Stadium,Rob Whitton,Hope Patricia Powell,Carla Ward,Home Win,Brighton & Hove Albion WFC,0,0,"[59, 60]",[],"[59, 60]",16,13,29,17,4,6,1,7,54.0,46.0,2,0,2,10,22,32,0,0,0,362,272,634,7,0,7,151,129,59,108,121,29,19.97,18.74,9,7,16,618,538,1156,0,4,4,0,0,0,[],"[65, 71, 81, 91]","[65, 71, 81, 91]",[],[],[]
2,3764230,2020-09-06,15:00:00.000,England - FA Women's Super League,2020/2021,Tottenham Hotspur Women,West Ham United LFC,1,1,1,Regular Season,The Hive Stadium,Kirsty Dowle,Karen Hills,Matt Beard,Draw,Draw,0,0,[],[57],[57],14,21,35,12,8,8,4,12,51.0,49.0,4,1,5,14,9,23,0,0,0,381,321,702,8,0,8,117,164,73,116,138,51,21.76,20.38,9,6,15,640,592,1232,0,1,1,0,0,0,[],[46],[46],[],[],[]
3,3764232,2020-09-06,15:30:00.000,England - FA Women's Super League,2020/2021,Manchester United,Chelsea FCW,1,1,1,Regular Season,Leigh Sports Village Stadium,Rebecca Welch,Casey Stoney,Emma Hayes,Draw,Draw,0,1,[79],[23],"[79, 23]",10,13,23,12,16,4,6,10,52.0,48.0,5,5,10,9,8,17,0,0,0,432,406,838,7,0,7,196,170,32,136,191,50,17.62,20.19,7,6,13,647,634,1281,2,0,2,0,0,0,"[76, 85]",[],"[76, 85]",[],[],[]
4,3764233,2020-09-06,13:30:00.000,England - FA Women's Super League,2020/2021,Arsenal WFC,Reading WFC,6,1,1,Regular Season,Meadow Park,Amy Fearn,Joseph Montemurro,Kelly Chambers,Home Win,Arsenal WFC,3,0,"[15, 33, 40, 63, 78, 81]",[89],"[15, 33, 40, 63, 78, 81, 89]",14,14,28,20,11,10,4,14,57.0,43.0,7,1,8,7,14,21,1,0,1,411,299,710,20,0,20,183,145,62,116,114,48,19.18,20.37,5,7,12,651,510,1161,2,3,5,0,0,0,"[41, 82]","[55, 60, 68]","[41, 82, 55, 60, 68]",[],[],[]
5,3764240,2020-09-06,15:00:00.000,England - FA Women's Super League,2020/2021,Bristol City WFC,Everton LFC,0,4,1,Regular Season,Twerton Park Stadium,Scott Jackson,Tanya Oxtoby,Willie Kirk,Away Win,Everton LFC,0,2,[],"[7, 19, 62, 90]","[7, 19, 62, 90]",14,11,25,10,14,3,7,10,36.0,64.0,2,12,14,10,17,27,0,1,1,224,494,718,3,0,3,91,90,30,212,192,63,18.68,18.76,3,6,9,425,702,1127,2,1,3,0,0,0,"[28, 48]",[78],"[28, 48, 78]",[],[],[]
6,3764238,2020-09-12,18:20:00.000,England - FA Women's Super League,2020/2021,West Ham United LFC,Arsenal WFC,1,9,2,Regular Season,Chigwell Construction Stadium,Abigail Byrne,Matt Beard,Joseph Montemurro,Away Win,Arsenal WFC,1,4,[27],"[23, 34, 40, 42, 52, 54, 56, 72, 86]","[27, 23, 34, 40, 42, 52, 54, 56, 72, 86]",12,10,22,3,25,2,19,21,39.0,61.0,1,3,4,16,9,25,